In [1]:
pip install fosforml==1.1.8

  Using cached https://files.pythonhosted.org/packages/94/2e/3613fd0ccdbf3709dec86f87fe7624737a6f08bd1a813c88e65e7352dfde/fosforml-1.1.8-py3-none-any.whl
  Found existing installation: fosforml 1.1.8a0
    Uninstalling fosforml-1.1.8a0:
      Successfully uninstalled fosforml-1.1.8a0
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from fosforml.model_manager.snowflakesession import get_session
session = get_session()

In [9]:
session.get_current_database()

'"FOSFORMLTEST"'

In [8]:
session.sql("USE DATABASE FOSFORMLTEST;").collect()

[Row(status='Statement executed successfully.')]

In [10]:
session.sql("USE SCHEMA PUBLIC;").collect()

[Row(status='Statement executed successfully.')]

In [11]:
df = session.table("HR_CHURN")

In [12]:
df.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECTS"  |"AVERAGE_MONTHLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT_COMPANY"  |"PROMOTION_LAST_5YEARS"  |"DEPARTMENT"  |"SALARY"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0.38                  |0.53               |2                  |157                      |3                     |False            |True            |False                    |sales         |low       |
|0.8                   |0.86               |5                  |262                      |6                     |False            |True            |False                    |sales         |medium 

In [13]:
from snowflake.snowpark.session import Session,col
import snowflake.snowpark.functions
from snowflake.ml.modeling.metrics import confusion_matrix, accuracy_score, f1_score, recall_score,precision_score
from snowflake.ml.modeling.preprocessing import LabelEncoder, StandardScaler
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.ensemble import GradientBoostingClassifier
from snowflake.ml.modeling.pipeline import Pipeline
import snowflake

In [14]:
def apply_label_encoding(input_data_frame):
    for i in input_data_frame.dtypes:
        if i[1].find('string') >= 0:
            label_encoder = LabelEncoder(input_cols=i[0],output_cols=i[0],drop_input_cols=True)
            input_data_frame = label_encoder.fit(input_data_frame).transform(input_data_frame)
    return input_data_frame

In [15]:
import os
print(os.getcwd())

/notebooks/notebooks


In [18]:
# import pandas as pd
# df = pd.read_csv("HR_CHURN.csv")
# snow_df = session.create_dataframe(df)
snow_df = df
input_data_frame, test_df = snow_df.randomSplit([0.75, 0.25])

In [19]:
label_encoder_column = []     
input_data_frame = apply_label_encoding(input_data_frame)
feature_cols = input_data_frame.columns
target_col = "SALARY"
feature_cols.remove(target_col)
OUTPUT_COLS = [target_col + '_PREDICTION']
input_data_frame.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECTS"  |"AVERAGE_MONTHLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT_COMPANY"  |"PROMOTION_LAST_5YEARS"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1.0       |7.0           |0.38                  |0.53               |2                  |157                      |3                     |False            |True            |False                    |
|2.0       |7.0           |0.8                   |0.86               |5                  |262                      |6                     |False            |True            |False                 

In [20]:
pipeline = GradientBoostingClassifier(input_cols=feature_cols, label_cols=target_col)
pipeline.fit(input_data_frame)

The version of package 'snowflake-snowpark-python' in the local environment is 1.25.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.


In [21]:
pred_snow_df = pipeline.predict(apply_label_encoding(test_df))

The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.


In [22]:
from fosforml import register_model

In [26]:
register_model(
  model_obj=pipeline,
  session=session,
  snowflake_df=pred_snow_df,
  name="HR_Churn_model",
  description="This is a test model",
  flavour="snowflake",
  dataset_name="Hr_churn",
    dataset_source = "HR_Churn",
  datasource_name="Hr_churn",
  model_type="classification",
  # conda_dependencies=["scikit-learn==1.3.2"],
  source="Notebook"
)

Calculating build time metrics

Progress: ██████████████                                                         20.0%
Calculating build time metrics

Progress: ████████████████████████████                                           40.0%


Calculating build time metrics

Progress: ██████████████████████████████████████████                             60.0%
Calculating build time metrics

Progress: ████████████████████████████████████████████████████████               80.0%
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


"Failed to register model 'MODEL_67843737_32D7_4468_9765_CF89E5B9C05E_FDC_HR_CHURN_MODEL'. (1300) (1304): 01b8a0bd-0712-3d30-0072-f3031401056a: 100357 (P0000): Python Interpreter Error:\nModuleNotFoundError: No module named 'sklearn.ensemble._gb_losses' in function CreateModule-6d50322e-57ea-4808-bc09-f03522455321 with handler predict.infer"